# Convert EPIC-Diff Data to HyperNeRF Style

In [1]:
import numpy as np
from pathlib import Path
import json
import imageio
import shutil

epic_dir = Path('data/EPIC-Diff')
hypernerf_dir = Path('data/hypernerf')
scene_id = 'P01_01'
epic_data_dir = epic_dir / scene_id
epic_rgb_dir = epic_data_dir / 'frames'
epic_mask_dir = epic_data_dir / 'annotations'

root_dir = hypernerf_dir / scene_id
# Where to save RGB images.
rgb_dir = root_dir / 'rgb' / '1x'
# Where to save mask images.
mask_dir = root_dir / 'mask' / '1x'
camera_dir = root_dir / 'camera'

rgb_dir.mkdir(exist_ok=True, parents=True)
mask_dir.mkdir(exist_ok=True, parents=True)
camera_dir.mkdir(exist_ok=True, parents=True)

mask_list = sorted(list(epic_mask_dir.glob('*.bmp')))
rgb_list = sorted(list(epic_rgb_dir.glob('*.bmp')))

with open(epic_data_dir / 'meta.json','r') as f:
    epic_meta = json.load(f)

epic_meta.keys()


dict_keys(['ids_all', 'ids_train', 'ids_test', 'ids_val', 'poses', 'nears', 'fars', 'images', 'intrinsics'])

In [3]:
epic_meta['fars']

{'22': 4.086954207670734,
 '23': 4.107134076166976,
 '24': 3.9466916982550972,
 '25': 3.7795458332857703,
 '29': 4.044048813839661,
 '30': 3.4796195776727337,
 '31': 4.1306283815136045,
 '32': 4.106296446876624,
 '33': 4.116114342909881,
 '34': 4.253852894036952,
 '35': 3.884302798763538,
 '36': 4.412573541777994,
 '37': 3.9029618020216486,
 '38': 3.879281898334316,
 '39': 3.842915997379642,
 '40': 3.9154192139344697,
 '41': 3.9485495307241996,
 '42': 3.954567059518391,
 '43': 3.9044369931098983,
 '44': 3.956390128275045,
 '45': 4.204855625038318,
 '46': 3.8774352464572175,
 '47': 3.9269628334225866,
 '48': 3.9338342704352955,
 '49': 3.9284207477209327,
 '50': 3.982522784659693,
 '51': 3.9174603296324446,
 '52': 4.277661487543875,
 '53': 4.348725217994217,
 '62': 4.148249297483869,
 '63': 3.9580656410618063,
 '64': 4.015997359490596,
 '65': 4.169496387577268,
 '66': 4.124947099623085,
 '67': 4.021065246606347,
 '68': 4.032438589373981,
 '69': 4.106380715460579,
 '70': 4.098721001157351

In [25]:
# copy all the images and masks
for mask in mask_list:
    # shutil.copy(str(mask), str(mask_dir))
    img = imageio.imread(str(mask))
    imageio.imsave(str(mask_dir / f'{mask.stem}.png'), img)
    

for rgb in rgb_list:
    # shutil.copy(str(rgb), str(rgb_dir))
    img = imageio.imread(str(rgb))
    imageio.imsave(str(rgb_dir / f'{rgb.stem}.png'), img)

image_size = [img.shape[1], img.shape[0]]


TypeError: 'tuple' object is not callable

In [54]:
# write cameras
for i, img_id in enumerate(epic_meta['ids_all']):
    meta = {}
    img_name = epic_meta['images'][str(img_id)].replace('.bmp', '')
    pose = np.array(epic_meta['poses'][str(img_id)])


    # orientation is the world to camera rotation matrix!!!
    meta['orientation'] = pose[:3,:3].T.tolist()
    meta['position'] = pose[:3,3].tolist()
    meta['focal_length'] = epic_meta['intrinsics'][0][0]
    meta['principal_point'] = [epic_meta['intrinsics'][0][2], epic_meta['intrinsics'][1][2]] 
    meta['skew'] = 0.
    meta['pixel_aspect_ratio'] = 1. # check
    meta['radial_distortion'] = [0., 0., 0.] # check
    meta['tangential_distortion'] = [0., 0.] # check
    meta['image_size'] = image_size

    with open(str(camera_dir / img_name) + '.json', 'w') as f:
        json.dump(meta, f, indent=2)

In [58]:
# write dataset split

all_ids = list(map(lambda x: epic_meta['images'][str(x)].replace('.bmp', ''), epic_meta['ids_all']))
train_ids = list(map(lambda x: epic_meta['images'][str(x)].replace('.bmp', ''), epic_meta['ids_train']))
actual_val_ids = list(map(lambda x: epic_meta['images'][str(x)].replace('.bmp', ''), epic_meta['ids_val']))
val_ids = list(map(lambda x: epic_meta['images'][str(x)].replace('.bmp', ''), epic_meta['ids_test']))

print(f'Num of train: {len(train_ids)}')
print(f'Num of val: {len(val_ids)}')

dataset_json = {
    'count': len(all_ids),
    'num_exemplars': len(train_ids),
    'ids': all_ids,
    'train_ids': train_ids,
    'val_ids': val_ids, # those are actuall test ids
    'actual_val_ids': actual_val_ids,
}

dataset_json_path = root_dir / 'dataset.json'
with dataset_json_path.open('w') as f:
    json.dump(dataset_json, f, indent=2)

print(f'Saved dataset information to {dataset_json_path}')

Num of train: 752
Num of val: 54
Saved dataset information to data/hypernerf/P01_01/dataset.json


In [59]:
# write meta info

# @title Save metadata information to `metadata.json`.
import bisect

metadata_json = {}
for i, image_id in enumerate(train_ids):
  metadata_json[image_id] = {
      'warp_id': i,
      'appearance_id': i,
      'camera_id': 0,
  }
for i, image_id in enumerate(val_ids):
  i = bisect.bisect_left(train_ids, image_id)
  metadata_json[image_id] = {
      'warp_id': i,
      'appearance_id': i,
      'camera_id': 0,
  }

for i, image_id in enumerate(actual_val_ids):
  i = bisect.bisect_left(train_ids, image_id)
  metadata_json[image_id] = {
      'warp_id': i,
      'appearance_id': i,
      'camera_id': 0,
  }

metadata_json_path = root_dir / 'metadata.json'
with metadata_json_path.open('w') as f:
    json.dump(metadata_json, f, indent=2)

print(f'Saved metadata information to {metadata_json_path}')



Saved metadata information to data/hypernerf/P01_01/metadata.json


In [70]:
# write scene info
nears = list(epic_meta['nears'].values())
fars = list(epic_meta['fars'].values())

# # scene_center = np.mean(bbox_corners, axis=0)
scene_scale = 1.0 / np.sqrt(np.sum(np.max(fars) - np.min(nears) ** 2))
scene_center = np.array([0., 0., 0.])

print(f'Scene Center: {scene_center}')
print(f'Scene Scale: {scene_scale}')


scene_json_path = root_dir /  'scene_gt.json'
with scene_json_path.open('w') as f:
  json.dump({
      'scale': scene_scale,
      'center': scene_center.tolist(),
      'near': np.min(nears),
      'far': np.max(fars)
  }, f, indent=2)

# print(f'Saved scene information to {scene_json_path}')

Scene Center: [0. 0. 0.]
Scene Scale: 0.44721367685263774


In [69]:
np.max(fars) - np.min(nears)

5.000000067338147